In [37]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager

In [38]:
notebook_name = "adipell_granel"
s3 = S3AssetManager(notebook_name=notebook_name)

In [39]:
base_path = "raw/honduras"
granel = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='sin_adipel_alien_engorde')
con = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_alien_engorde')
con_punto = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_punto')

sultana =  s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='la_sultana_con')
granel.head()   

Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx
Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell Honduras y Salvador.xlsx


,fecha,kg producidos,Hora inicio,Hora final,Total minutos,Ton / h,Amperaje,Presión vapor (bar),H harina,H harina acondicionada,H producto terminad,T harina,T acondicionamiento,T prensa,T pellet,T ambiental,PDI,finos < 1mm,dieta,producto
0,2025-08-12,45670,03:59:00,06:25:00,146,18.768493,654.0,2.7,0.116,0.154,0.129,34.8,70.0,78.6,34.1,32.0,87.00,0.011,Aliengorde 2,sin adipell
1,2025-08-15,30000,04:50:00,06:00:00,70,25.714286,NaN,2.8,0.126,0.166,0.120,34.1,72.0,80.7,35.1,32.1,86.00,0.019,Aliengorde 2,sin adipell
2,2025-08-17,47110,02:46:00,05:26:00,160,17.666250,606.0,3.0,0.125,0.157,0.128,34.6,58.0,59.0,34.5,31.5,87.00,0.007,Aliengorde 2,sin adipell
3,2025-08-18,22300,07:27:00,08:37:00,70,19.114286,510.0,2.9,0.124,0.160,0.123,33.2,72.0,79.8,NaN,32.6,87.00,0.010,Aliengorde 2,sin adipell
4,NaT,24085,15:14:00,15:59:00,45,32.113333,625.0,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.32,NaN,Aliengorde 2,con adipell


In [40]:
granel.columns = granel.columns.str.strip()
granel["fecha"] = pd.to_datetime(granel["fecha"])
granel["rendimiento"] = granel["kg producidos"]/(granel["Total minutos"]/60)

In [41]:
granel_group = granel.groupby("producto").agg(
    toneladas_producidas=('kg producidos', 'sum'),
    tiempo_min_produccion=('Total minutos', 'sum'),
    rendimiento=('rendimiento', 'mean'),
    amperaje_promedio=('Amperaje', "mean"),
    presion_promedio=('Presión vapor (bar)', 'mean'),
    humedad_prom_harina=('H harina', "mean"),
    humedad_prom_harina_acond=('H harina acondicionada', "mean"),
    humedad_prom_harina_termidado=('H producto terminad', "mean"),
    pdi=('PDI', "mean"),
    finos=('finos < 1mm', "mean"),
).reset_index()
#granel_group["tiempo_horas_produccion"] = granel_group["tiempo_min_produccion"]/60
granel_group["toneladas_producidas"] = granel_group["toneladas_producidas"]/1000
granel_group["rendimiento"] = granel_group["rendimiento"]/1000
granel_group["experimento"] = "Exp II"

granel_group

,producto,toneladas_producidas,tiempo_min_produccion,rendimiento,amperaje_promedio,presion_promedio,humedad_prom_harina,humedad_prom_harina_acond,humedad_prom_harina_termidado,pdi,finos,experimento
0,con adipell,24.085,45,32.113333,625.0,3.10,NaN,NaN,NaN,95.32,NaN,Exp II
1,sin adipell,145.080,446,20.315829,590.0,2.85,0.12275,0.15925,0.125,86.75,0.01175,Exp II


In [42]:

def stripplot_con_medias_pro(df, categoria: str, values: str, title: str, filename: str = None):
    """
    Strip plot profesional:
      - Puntos (semi-transparencia)
      - Boxplot translúcido (mediana + IQR)
      - Media ± IC95% por categoría
      - Línea de media global
      - Conteos n por categoría
      - Colores corporativos y orden fijo
    """

    # Orden de categorías y colores corporativos
    orden_categorias = ["dado pellet", "enfriador", "envasado"]
    colores_corporativos = {
        "dado pellet": "#1C8074",
        "enfriador": "#666666",
        "envasado": "#94AF92"
    }

    # Asegurar orden categórico
    df = df.copy()
    df[categoria] = pd.Categorical(df[categoria], categories=orden_categorias, ordered=True)

    # Estadísticas por categoría
    g = df.groupby(categoria, observed=True)[values]
    stats = g.agg(["count", "mean", "median", "std"]).reindex(orden_categorias)
    stats["sem"] = stats["std"] / np.sqrt(stats["count"].clip(lower=1))
    stats["ci95"] = 1.96 * stats["sem"]

    # Media global
    mean_global = df[values].mean()

    # 1) Strip plot (puntos)
    fig = px.strip(
        df,
        x=categoria, y=values, color=categoria,
        category_orders={categoria: orden_categorias},
        color_discrete_map=colores_corporativos,
        template="plotly_white"
    )
    # Afinar estilo de los puntos
    for tr in fig.data:
        tr.update(
            marker=dict(size=7, opacity=0.6, line=dict(width=0)),
            hovertemplate=f"<b>{values}</b>: %{{y:.2f}}<br>{categoria}: %{{x}}<extra></extra>"
        )

    # 2) Boxplot translúcido detrás (misma paleta)
    for cat in orden_categorias:
        sub = df.loc[df[categoria] == cat, values]
        if sub.empty:
            continue
        fig.add_trace(
            go.Box(
                x=[cat]*len(sub), y=sub,
                name=f"{cat} (IQR)",
                marker_color=colores_corporativos[cat],
                line=dict(color=colores_corporativos[cat]),
                fillcolor="rgba(0,0,0,0)",  # sin relleno sólido
                opacity=0.25,
                boxmean=False,
                showlegend=False
            )
        )

    # 3) Media ± IC95% por categoría (marcador + barras de error)
    fig.add_trace(
        go.Scatter(
            x=stats.index.tolist(),
            y=stats["mean"],
            mode="markers",
            name="Media ± IC95%",
            marker=dict(symbol="diamond", size=12, color="black"),
            error_y=dict(type="data", array=stats["ci95"], thickness=1.5, width=6, color="black"),
            hovertemplate="<b>Media</b>: %{y:.2f}<br>IC95%: ±%{customdata:.2f}<extra></extra>",
            customdata=stats["ci95"].values,
            showlegend=True
        )
    )

    # 4) Línea de media global
    fig.add_hline(
        y=mean_global,
        line=dict(color="black", width=1, dash="dot"),
        annotation_text=f"Media global: {mean_global:.2f}",
        annotation_position="top right",
        annotation_font_color="black"
    )

    # 5) Conteos n por categoría (debajo del eje X)
    for i, cat in enumerate(orden_categorias):
        n = int(stats.loc[cat, "count"]) if pd.notna(stats.loc[cat, "count"]) else 0
        fig.add_annotation(
            x=cat, y=df[values].min(),
            yshift=-30, showarrow=False,
            text=f"n={n}", font=dict(color="black", size=11)
        )

    # Layout pro
    fig.update_layout(
        title=title,
        xaxis_title=categoria,
        yaxis_title=values,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        margin=dict(t=80, r=30, b=80, l=60),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
    )
    fig.update_xaxes(showline=True, linecolor="black", mirror=True)
    fig.update_yaxes(showline=True, linecolor="black", mirror=True, gridcolor="rgba(0,0,0,0.1)")

    if filename:
        fig.write_html(filename, include_plotlyjs="cdn", full_html=True)

    return fig


In [43]:
granel_group

,producto,toneladas_producidas,tiempo_min_produccion,rendimiento,amperaje_promedio,presion_promedio,humedad_prom_harina,humedad_prom_harina_acond,humedad_prom_harina_termidado,pdi,finos,experimento
0,con adipell,24.085,45,32.113333,625.0,3.10,NaN,NaN,NaN,95.32,NaN,Exp II
1,sin adipell,145.080,446,20.315829,590.0,2.85,0.12275,0.15925,0.125,86.75,0.01175,Exp II


In [44]:
fig_pdi =plot_bar(
df=granel_group,
x_col="experimento",
y_col="pdi",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='delta',
#title="PDI (%)",
)
fig_pdi.show()
s3.save_plotly_html(fig_pdi, "pdi_granel.html")

In [45]:
fig_rendimiento =plot_bar(
df=granel_group,
x_col="experimento",
y_col="rendimiento",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="Rendimiento (Ton/H)",
)
fig_rendimiento.show()
s3.save_plotly_html(fig_rendimiento, "rendimiento_granel.html")

In [46]:
fig_final = create_subplot_grid(
    figures=[fig_rendimiento, fig_pdi],
    rows=1, cols=2,
    titles=["Rendimiento (Ton/H)", "PDI (%)"],
    main_title="Métricas de rendimiento y calidad",
    shared_y=False,
    width=1000, height=400
)

fig_final.show()
s3.save_plotly_html(fig_final, "pdi_finos_rendimiento_granel.html")

In [47]:
fig_amperaje_promedio = plot_bar(
df=granel_group,
x_col="experimento",
y_col="amperaje_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="Rendimiento (Ton/H)",
)
fig_amperaje_promedio.show()
s3.save_plotly_html(fig_amperaje_promedio, "amperaje_promedio_granel.html")

In [48]:
fig_presion_promedio =plot_bar(
df=granel_group,
x_col="experimento",
y_col="presion_promedio",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="Rendimiento (Ton/H)",
)
fig_presion_promedio.show()
s3.save_plotly_html(fig_presion_promedio, "presion_promedio_granel.html")

In [49]:
fig_final = create_subplot_grid(
    figures=[fig_amperaje_promedio, fig_presion_promedio],
    rows=1, cols=2,
    titles=["Amperaje promedio (Amp)", "Presion promedio (Bar)"],
    main_title="Comparativo parámetros de control",
    shared_y=False,
    width=1000, height=400
)

fig_final.show()
s3.save_plotly_html(fig_final, "amp_presion_granel.html")

In [50]:
# 1. Definir tu configuración de negocio fuera de la función
config_colores = {
    "dado pellet": "#1C8074",
    "enfriador": "#666666",
    "envasado": "#94AF92"
}
orden_pasos = ["dado pellet", "enfriador", "envasado"]

fig = plot_statistical_strip(
    df=con_punto,
    x_col="punto",      # Tu columna 'categoria'
    y_col="PDI",      # Tu columna 'values'
    category_order=orden_pasos, # Inyectamos el orden
    color_map=config_colores,   # Inyectamos los colores
    title="Análisis de PDI por Punto",
    y_title="PDI (%)"
)

fig.show()
s3.save_plotly_html(fig_final, "stripplot_pdi_orden.html")

In [51]:
fig = plot_statistical_strip(
    df=con_punto,
    x_col="punto",      
    y_col="Finos (malla 18)",    
    category_order=orden_pasos,
    color_map=config_colores,
    title="Análisis de Finos por Punto",
    y_title="Finos (malla 18)"
)

fig.show()
s3.save_plotly_html(fig_final, "stripplot_finos_18_orden.html")

In [52]:
fig = plot_statistical_strip(
    df=con_punto,
    x_col="punto",      
    y_col="Retencion Malla 6",    
    category_order=orden_pasos,
    color_map=config_colores,
    title="Análisis de Finos por Punto",
    y_title="Retencion (malla 6)"
)

fig.show()
s3.save_plotly_html(fig, "stripplot_finos_6_orden.html")